### 📌 Agentic RAG
- **기존 RAG**: `Retriever → LLM` 직렬 파이프라인
- **Agentic RAG**: LLM이 스스로 Tool 호출(검색, 리랭킹 등) 결정을 내림
- 장점:
  - 질문 유형에 따라 “검색” vs “직접 답변” 선택 가능
  - 검색 결과가 부족하면 여러 번 툴 호출 가능
- 핵심 구성:
  - Retriever를 Tool로 등록
  - LLM Agent가 Tool을 호출해 증거 문서 확보

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [3]:
# Agentic RAG: 검색기 + 리랭커 + LLM Agent 조합
# pip install langchain langchain-openai langchain-community faiss-cpu

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.agents import initialize_agent, Tool

# 1) LLM, 임베딩 준비
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
embeddings = OpenAIEmbeddings()

# 2) 문서 준비 (간단 예시)
docs = [
    "LangChain은 LLM 기반 앱 개발을 쉽게 하는 프레임워크다.",
    "RAG는 검색과 생성을 결합하는 접근 방식이다.",
    "Agent는 도구를 사용하여 복잡한 작업을 수행할 수 있다."
]

vectorstore = FAISS.from_texts(docs, embeddings)
retriever = vectorstore.as_retriever()

# 3) 검색기를 Agent의 Tool로 바인딩
def retriever_tool(query: str):
    results = retriever.get_relevant_documents(query)
    return "\n".join([d.page_content for d in results])

tools = [
    Tool(
        name="KnowledgeBase",
        func=retriever_tool,
        description="RAG용 문서 검색을 수행한다."
    )
]

# 4) Agent 초기화
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

# 5) 실행
agent.run("LangChain과 RAG의 관계를 설명해줘")

C:\Users\james\AppData\Local\Temp\ipykernel_19000\3851514298.py:36: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
C:\Users\james\AppData\Local\Temp\ipykernel_19000\3851514298.py:39: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` 



> Entering new AgentExecutor chain...
I should use the KnowledgeBase to search for information on the relationship between LangChain and RAG.
Action: KnowledgeBase
Action Input: "Relationship between LangChain and RAG"

C:\Users\james\AppData\Local\Temp\ipykernel_19000\3851514298.py:24: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)



Observation: RAG는 검색과 생성을 결합하는 접근 방식이다.
LangChain은 LLM 기반 앱 개발을 쉽게 하는 프레임워크다.
Agent는 도구를 사용하여 복잡한 작업을 수행할 수 있다.
Thought:Based on the information gathered, LangChain is a framework that makes it easy to develop apps based on LLM, while RAG is an approach that combines search and generation. LangChain may be used by agents to perform complex tasks. 
Final Answer: LangChain is a framework for LLM-based app development, while RAG is an approach that combines search and generation.

> Finished chain.


'LangChain is a framework for LLM-based app development, while RAG is an approach that combines search and generation.'

### 📌 CSV/Excel 데이터분석 Agent
- LangChain은 **DataFrame Agent**를 제공 → LLM이 판다스 연산을 수행
- 사용 절차:
  1. CSV/Excel을 Pandas DataFrame으로 로드
     - `pd.read_csv("파일.csv")`
     - `pd.read_excel("파일.xlsx")`
  2. `create_pandas_dataframe_agent(llm, df)`
  3. 자연어 질문 → Agent가 내부적으로 Python 코드 작성 & 실행
- 장점:
  - 데이터 통계/시각화/그룹 연산 등을 자연어로 질의
- 주의:
  - 민감 데이터 사용 시 보안 고려
  - LLM이 잘못된 코드 생성 시 에러 발생 가능

In [7]:
# pip install -U langchain-experimental pandas openpyxl

import pandas as pd
from langchain_openai import ChatOpenAI
from langchain_experimental.agents import create_pandas_dataframe_agent

# 1) 예시 데이터프레임
df = pd.DataFrame({
    "이름": ["철수", "영희", "민수", "지영"],
    "나이": [23, 25, 21, 29],
    "점수": [88, 92, 77, 85]
})

# 2) LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# 3) DataFrame Agent 생성 (⚠️ 코드 실행 동의)
agent = create_pandas_dataframe_agent(
    llm,
    df,
    verbose=True,
    allow_dangerous_code=True,       # ← 반드시 명시
    include_df_in_prompt=True,       # 모델에 상단 n행을 보여줄지 여부
    number_of_head_rows=5,           # 상단 몇 행을 보여줄지
    # max_iterations=8,              # 필요 시 반복 제한
    # return_intermediate_steps=True # 중간 단계 디버깅용
)

# 4) 실행 (.run 또는 .invoke 모두 가능)
print(agent.run("평균 나이는 몇 살이야?"))
print(agent.run("점수가 가장 높은 학생의 이름은 누구야?"))



> Entering new AgentExecutor chain...
Thought: To calculate the average age, we need to find the mean of the '나이' column in the dataframe.
Action: python_repl_ast
Action Input: df['나이'].mean()24.5The average age is 24.5 years old.
Final Answer: 24.5 years

> Finished chain.
24.5 years


> Entering new AgentExecutor chain...
Thought: We need to find the student with the highest score.
Action: [python_repl_ast]
Action Input: df[df['점수'] == df['점수'].max()]['이름'].values[0][python_repl_ast] is not a valid tool, try one of [python_repl_ast].I will use the python_repl_ast tool instead.
Action: [python_repl_ast]
Action Input: df[df['점수'] == df['점수'].max()]['이름'].values[0][python_repl_ast] is not a valid tool, try one of [python_repl_ast].I will use the python_repl_ast tool instead.
Action: [python_repl_ast]
Action Input: df[df['점수'] == df['점수'].max()]['이름'].values[0][python_repl_ast] is not a valid tool, try one of [python_repl_ast].I will use the python_repl_ast tool instead.
Action: [pytho

# ✅ 최종 정리
- **Agentic RAG**: Retriever를 Tool로 등록 → Agent가 필요시 검색/활용
- **CSV/Excel 분석 Agent**: DataFrame을 Agent에게 넘겨 자연어 질의로 분석

👉 Agent를 활용하면 단순 QA를 넘어 **동적 검색·분석 워크플로우**를 구성할 수 있음.
